In [2]:
import google.genai as genai
import os
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
from pydantic import BaseModel
from typing import List, Dict

class Item(BaseModel):
    item: str
    quantity: str

class RecipeResponse(BaseModel):
    content: str
    items: List[Item]

In [25]:
def create_content_generation_prompt(role, work, additional_info=None):
    prompt = f'''
    You are an expert {role}.
    This time you have to do the following:
    {work}
    Generate content based on it. It should be simple, straight forward and easy to implement if implementation is possible.
    If there are any items present in the content generated, please keep a track of it and its quantity (start with smallest possible available).
    Please do not use any filler words and no **highlighted headers and titles**.
    Output should be strictly in the following JSON structure:
    ```json{{
    content : generated content,
    items : [items]
    }}```
    '''
    if additional_info is not None:
        prompt = prompt + '\n' + f'''
        Also, consider this additional details:
        {additional_info}.
        '''

    return prompt

In [26]:
client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=create_content_generation_prompt('chef', 'Make egg fried rice for 4 people', additional_info='Add indian spices.')
)

In [27]:
print(response.candidates[0].content.parts[0].text)

```json
{
  "content": "## Egg Fried Rice (Indian Style) for 4\n\nThis recipe combines classic egg fried rice with warm Indian spices for a flavorful twist.\n\n**Ingredients:**\n\n*   2 cups cooked rice (preferably day-old, cooled)\n*   2 eggs\n*   1/2 medium onion, finely chopped\n*   1 green chili, finely chopped (adjust to taste)\n*   1/2 teaspoon ginger-garlic paste\n*   1/4 teaspoon turmeric powder\n*   1/2 teaspoon red chili powder\n*   1/2 teaspoon garam masala\n*   1 tablespoon vegetable oil\n*   1/4 cup chopped cilantro (coriander leaves)\n*   Salt to taste\n*   1/4 teaspoon black pepper\n\n**Instructions:**\n\n1.  Heat oil in a wok or large frying pan over medium-high heat.\n2.  Scramble eggs with a pinch of salt and black pepper. Cook until set, then remove from pan and set aside.\n3.  Add chopped onion and green chili to the same pan and saute until onions are translucent, about 2-3 minutes.\n4.  Add ginger-garlic paste and saute for another minute until fragrant.\n5.  Stir

In [41]:
import json

def clean_json_output(raw_text):
    """ Extract valid JSON from Gemini's markdown-styled output. """
    try:
        # Remove Markdown ```json ... ``` wrapper
        if raw_text.startswith("```json"):
            raw_text = raw_text.strip("```json").strip("```")
        return json.loads(raw_text)
    except json.JSONDecodeError as e:
        print(f"JSON Decoding Error: {e}")
        return None

In [ ]:
raw_json = clean_json_output(response.text)
# print(raw_json)
# print(type(raw_json))
try:
    validated_response = RecipeResponse.model_validate(raw_json)
    x = validated_response.model_dump()
    print(x['content'])
    print(x['items'])
except Exception as e:
    print(f"Error in JSON response: {e}")

## Egg Fried Rice (Indian Style) for 4

This recipe combines classic egg fried rice with warm Indian spices for a flavorful twist.

**Ingredients:**

*   2 cups cooked rice (preferably day-old, cooled)
*   2 eggs
*   1/2 medium onion, finely chopped
*   1 green chili, finely chopped (adjust to taste)
*   1/2 teaspoon ginger-garlic paste
*   1/4 teaspoon turmeric powder
*   1/2 teaspoon red chili powder
*   1/2 teaspoon garam masala
*   1 tablespoon vegetable oil
*   1/4 cup chopped cilantro (coriander leaves)
*   Salt to taste
*   1/4 teaspoon black pepper

**Instructions:**

1.  Heat oil in a wok or large frying pan over medium-high heat.
2.  Scramble eggs with a pinch of salt and black pepper. Cook until set, then remove from pan and set aside.
3.  Add chopped onion and green chili to the same pan and saute until onions are translucent, about 2-3 minutes.
4.  Add ginger-garlic paste and saute for another minute until fragrant.
5.  Stir in turmeric powder, red chili powder and garam m